In [1]:
import pandas as pd
import joblib
from gensim. models import LdaMulticore, CoherenceModel

In [2]:
df=pd.read_excel('wipo_processed_phrases.xlsx')
df.head()

,Application Id,Application Year,Full Text,Applicant,type,clean_text,phrases
0,US42597370,2000,bacterial strains genes and enzymes for contro...,Institute Of Molecular Agrobiology,Research Institute,bacterial strain gene enzyme control bacterial...,"['bacterial_strain', 'gene', 'enzyme', 'contro..."
1,US39458207,2000,reversible rolling method and reversible rolli...,Hitachi,Company,reversible rolling reversible rolling pass sch...,"['reversible', 'rolling', 'reversible', 'rolli..."
2,WO2001024205,2000,composite magnetic sheet and method of produci...,Tokin Corporation,Company,composite magnetic sheet composite magnetic bo...,"['composite', 'magnetic', 'sheet', 'composite'..."
3,US39488862,2000,compounding assembly for nutritional fluids an...,Baxter International Inc,Company,compounding assembly nutritional fluid assembl...,"['compounding', 'assembly', 'nutritional', 'fl..."
4,US39579616,2000,hydrotreating catalyst for hydrotreating hydro...,Tonen Corporation,Company,hydrotreating catalyst hydrocarbon highactivit...,"['hydrotreating_catalyst', 'hydrocarbon', 'hig..."


In [3]:
import ast
from gensim import corpora
from tqdm import tqdm
import numpy as np

np.random.seed(3)

tokens=[ast.literal_eval(x) for x in df.drop_duplicates('Full Text')['phrases'].tolist()]

id2w = corpora.Dictionary(tokens)
corpus=[]
for rev in tqdm(tokens):
    corpus.append(id2w.doc2bow(rev))
len(tokens)

100%|██████████| 89250/89250 [00:02<00:00, 33480.61it/s]


89250

In [ ]:
import time
model={}
coherence_model_lda=[]
for t in tqdm(range(6,15)):
    model[t] = LdaMulticore(corpus, num_topics=t, id2word=id2w, passes=50,workers=15)
    joblib.dump(model[t], f'model_{t}.sav')
    vis[t] = pyLDAvis.gensim.prepare(model[t], corpus, dictionary=id2w)
    coherence_model_lda.append(CoherenceModel(model=model[t], texts=tokens, dictionary=id2w, coherence='c_v').get_coherence())
    if t%2==0:
        time.sleep(5)

 50%|█████     | 2/4 [20:03<20:09, 604.84s/it]

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=list(range(16,20)), y=coherence_model_lda, mode='lines+markers'))
fig.update_layout(template='simple_white')
fig.show()